**Accessibility and playing around**
Inspiration for this code comes from:

- Rafa Pereira [github page]()
- `tracc` [Python library](https://github.com/jamaps/tracc) quoted from the above

UH! I found a notebook project that does it, let's see if useful:
https://github.com/jamaps/tracc/blob/master/examples/basic_accessibility_example.ipynb